### model.save()
- 학습의 결과를 저장하기 위한 함수
- 모델 형태(architecture,  cnn 이라던가,,)와 parameter를 저장
- 학습 중간과정의 저장을 통해 최선의 결과모델 선택
- 외부 연구자와 공유
<br><br>
- model.state_dict() : 모델의 파라미터
- .pt : pytorch에서 모델을 저장할 때 사용하는 확장자

#### parameter만 저장하고 load
- 모델이 동일 한 형태일 때 
- 일반적으로 많이 사용됨 

In [ ]:
import torch
import os
# Print model's tstate_dict
print("model's state_dict:")
for param_tensor in model.state_dict(): #state_dict : 모델의 파라메터를 표시
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())
# 모델의 파라메터를 저장
torch.save(model.state_dict(),
            # pytorch에서 모델을 저장할 때는 .pt 확장자 사용
            os.path.join(MODEL_PATH,"model.pt"))

new_model = TheModelClass()
# 같은 모델의 형태가 보장될 때, parameter만 불러온다. load_state_dict 사용
new_model.load_state_dict(torch.load(os.path.join(MODEL_PATH,"model.pt")))


#### 모델 architecture 함께 저장
- 보통 많이 사용 안됨
- 모델은 코드로 공유 되기 때문.
- 결과를 공유할 때는 parameter만 저장하고 전달.

In [ ]:

# 모델 architecture 와 함께 저장
torch.save(model,os.path.join(MODEL_PATH,"model.pt"))
# 모델의 architecture와 함께 load
model = torch.load(os.path.join(MODEL_PATH,"model.pt"))

#### checkpoints
- 학습의 중간 결과를 저장하여 최선의 결과를 선택
- earlystopping 기법 사용시 이전 학습의 결과물 저장
- loss와 metric 값을 지속적으로 확인 저장
- 일반적으로, epoch, loss, metric을 함께 저장하여 저장

In [ ]:
# dict 타입으로 모델의 정보를 epoch과 함께 넣어주어 저장함
from pickletools import optimize


torch.save({
        'epoch': e,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': epoch_loss,
        # 파일명에 epoch_loss, epoch_acc를 명시
        }, f"saved/checkpoint_model_{e}_{epoch_loss/len(dataloader)}_{epoch_acc/len(dataloader)}.pt")

#dict 처럼 접근해서 골라서 쓰면 된다.
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoc']
loss = checkpoint['loss']

## Transfer learning
- 다른 데이터셋으로 만든 모델을 현재 데이터에 적용(pretrained model 활용)
- 일반적으로 대용량 데이터셋으로 만들어진 모델의 성능이 좋다
- 현재의 DL에선 가장 일반적인 학습 기법
- backbone architecture가 잘 학습된 모델에서 일부분만 변경하여 학습을 수행(fine tuning)
    #### TorchVison에서 다양한 기본 모델 제공

    

### Freezing
- pretrained model 활용시, 모델의 일부분을 freeze 시킴
- 특정 위치의 parameter를 얼려버리는 것


vgg16 모델을 vgg에 할당하기

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#pretrained = True 해줘야 제대로 학습된 모델을 가져옴
#to(device) : 저 device를 사용하여 연산하고 거기에 저장할거라는뜻.
vgg = models.vgg16(pretrained = True).to(device)

In [ ]:
# 위랑 관계없음. 그냥 코드만 보라.

class MyNewNet(nn.Module):   
    def __init__(self):
        super(MyNewNet, self).__init__()
        self.vgg19 = models.vgg19(pretrained=True)
        # 내 데이터에 맞게 레이어들 추가.
        # 개냐 고양이냐 를 맞추는 하나의 output이 되도록.
        self.linear_layers = nn.Linear(1000, 1)


    # Defining the forward pass    
    def forward(self, x):
        x = self.vgg19(x)        
        return self.linear_layers(x)

# freezing 시키는것. 학습하지 않도록.
for param in my_model.parameters():
    param.requires_grad = False

# 학습시킬 새로운 layer만 required_grad = True 준다.
for param in my_model.linear_layers.parameters():
    param.requires_grad = True    

#### validation/testing 할때 model.eval()
#### training 할때, model.train() 해주자.
my_model.eval()